# Examples

This notebook demonstrates the SAS package documentation generation macros:
- `%collectfiles` - Scan and collect file information from directories
- `%generatemd` - Generate markdown documentation from source code comments  
- `%packagedoc` - Generate source for Jupyter book(MyST) and Generate document from package(zip).

We will create sample files in the WORK directory and demonstrate each macro's functionality.

## Prerequisites: Load SPF and Documentation Macros

First, load the SAS Packages Framework (SPF) and the documentation macros.

In [ ]:
/* Setup SPF - Load the SAS Packages Framework */
/* Adjust the path to where SPF is installed on your system */
%let pkgLocation=/path/to/packages;  /* Change this to your packages and SPF location */

filename packages "&pkgLocation.";  

/* Load the SPF macros */
%include packages(SPFinit.sas);

/* install package */
%installPackage(SASDoGs, github=k-nkmt);

/* Load the sasdogs package (documentation generator) */
%loadPackage(SASDoGs);

## Setup: Create Sample Files

We'll create two types of sample files:
1. **Package file** - Using SPF standard help comment format `/*** HELP START ***/`
2. **Custom file** - Using custom help comment format `/*--- HELP START ---*/`

In [ ]:
/* Get WORK directory path */
%let workdir = %sysfunc(pathname(work));

/* Create directory for SPF package structure */
%let pkgDir = &workdir./mypackage;
%let pkgMacroDir = &pkgDir./macro;

/* Create directories */
options dlcreatedir;
libname _pkg "&pkgDir.";
libname _pkg clear;
libname _pkg "&pkgMacroDir.";
libname _pkg clear;

/* Create directory for custom files */
%let customDir = &workdir./custom_docs;
libname _cust "&customDir.";
libname _cust clear;


/* Create package description file */
filename pkgdesc "&pkgDir./description.sas";

data _null_;
  file pkgdesc;
  put 'Type: Package' ;
  put 'Package: PackageName' ;
  put 'Title: A title/brief info for log note about your packages.' ;
  put 'Version: X.Y' ;
  put 'Author: Firstname1 Lastname1 (xxxxxx1@yyyyy.com)' ;
  put 'Maintainer: Firstname Lastname (xxxxxx@yyyyy.com)' ;
  put 'License: MIT' ;
  put 'Encoding: UTF8' ;
  put ' ' ;
  put 'Required: "Base SAS Software"' ;      
  put 'DESCRIPTION START:' ;
  put '  Xxxxxxxxxxx xxxxxxx xxxxxx xxxxxxxx xxxxxxxx. Xxxxxxx' ;
  put 'DESCRIPTION END:' ;
run;

/* Create package macro file with SPF standard help format */
filename pkgmac "&pkgMacroDir./hello.sas";

data _null_;
  file pkgmac;
  put '/*** HELP START ***/';
  put 'Say Hello to someone.';
  put '';
  put '## Parameters';
  put '';
  put '- **to=** - to whom to say hello';
  put '';
  put '## Example';
  put '';
  put '```sas';
  put '%hello(to=World)';
  put '```';
  put '';
  put '/***HELP END ***/';
  put '';
  put '%macro hello(to=);';
  put '  %put Hello &to..;';
  put '%mend hello;';
run;

/* Create custom file with custom help comment format */
filename custfile "&customDir./yourstyle.sas";

data _null_;
  file custfile;
  put '/*--- HELP START ---*/';
  put 'Comment.' ;
  put '/*--- HELP END ---*/';
run;

## Example 1: Using `%collectfiles` with Custom Help Format

Collect file information from the custom directory and generate documentation with custom help comment patterns.

In [4]:
/* Collect files from custom directory */
%collectfiles(
  targetLocation=&customDir.,
  listDataSet=work.custom_files
);

data work.print_ds ;
  set work.custom_files;
  path = transtrn(path, "&workdir.", 'workpath');
  base = transtrn(base, "&workdir.", 'workpath');
run;

title1 "Collected Files from Custom Directory";
proc print data=work.print_ds ;
  /* path contains full path, base contains full path of root */
run;
title;

OBS,path,base,name,folder,fileshort,ext,is_dir,depth
1,workpath/custom_docs,workpath/custom_docs,,,,,1,0
2,workpath/custom_docs/yourstyle.sas,workpath/custom_docs,yourstyle.sas,,yourstyle,sas,0,1


## Example 2: Using `%generatemd` with Custom Help Patterns (depth=0)

Generate a single consolidated markdown file from custom files using custom help comment delimiters.

In [ ]:
/* Create output directory */
%let customDocsOut = &workdir./custom_output;
libname _out "&customDocsOut.";
libname _out clear;

/* Generate markdown with custom help patterns */
%generatemd(
  fileList=work.custom_files,
  depth=0,
  docname=custom_docs.md,
  docsLocation=&customDocsOut.,
  startPtn="\/\*-+ HELP START -+\*\/",
  endPtn="\/\*-+ HELP END -+\*\/"
);

In [6]:
/* View the generated markdown file */
title1 "Generated Custom Documentation";
data _null_;
  infile "&customDocsOut./custom_docs.md" ;
  file print ;
  input ;
  put _infile_;
run;
title;

## Example 3: Using `%packagedoc` with MYST Engine (depth=2)

Generate source files for [Jupyter Book (MyST)](https://jupyterbook.org/) with individual files per source (depth=2).  
Configuration files (myst.yml, conf.py) may be edited after initial creation, so by default they are not overwritten (except for date and version information).  
To create them from scratch, set `newConf=1`.


In [ ]:
/* Create output directory for package documentation */
%let pkgDocsOut = &workdir./docs;
libname _pkgout "&pkgDocsOut.";
libname _pkgout clear;

/* Generate souce for MyST engine */
%packagedoc(
  filesLocation=&pkgDir.,
  docsLocation=&pkgDocsOut.,
  docDepth=2,
  engine=MYST,
  newConf=1
);


In [8]:
/* View the generated myst.yml configuration */
title1 "Generated myst.yml Configuration for Package Documentation";
data _null_;
  infile "&pkgDocsOut./myst.yml" ;
  file print ;
  input ;
  put _infile_;
  if _n_ = 5 then stop;
run;
title;

## Example 4: Using `%packagedoc` with Package File 

When package documentation is not publicly available or you don't know where it's published, try using `%helpPackage` first.  
If you find it cumbersome to use frequently, or if you want to obtain and create markdown or richer format documentation from package information, this functionality is available.


In [9]:
/* install sample package */
/* %installpackage(SQLinDS) ; */

%packagedoc(
  filesLocation=&pkgLocation./sqlinds.zip,
  docsLocation=&pkgLocation.
);

The SHA256 hash digest for package SQLinDS:,Type of hash digestF = fileC = content
F*CEAA4C90515F6E8AACBFFD55ABA6544E399EDBE0A7081107B62DCEE6F5430A1D,F
C*BD9C8A88831541082BEFC07954D5CDB4A6827D1A7902B911221AC6FE712A087E,C


In [10]:
/* View the generated sqlinds.md */
title1 "Generated sqlinds.md (Only not empty line is printed)";
data _null_;
  infile "&pkgLocation./sqlinds.md" ;
  file print ;
  input ;
  put _infile_;
  if _n_ = 5 then stop;
run;
title;